## Lab 6: SQL


### Starting Up SQL

Before we look at SQL syntax in detail, let's first get ourselves set up to run SQL queries in Jupyter.

#### Approach #1: SQL Magic

**1. Load the `sql` Module.** 

Load `%%sql` cell magic.

In [ ]:
%load_ext sql

**2. Connect to a database.**  

Here, we connect to the SQLite database `basic_examples.db`.

In [ ]:
%%sql
sqlite:///data/basic_examples.db

<br/>

**3. Run a simple SQL query.** 

Note the `%%sql` lets Jupyter parse the rest of the lines as a SQL command.

In [ ]:
%%sql
SELECT * FROM Dragon;

**Simple query, this time on two different lines.**

In [ ]:
%%sql
SELECT *
FROM Dragon;

#### Approach #2: `pd.read_sql`

It turns out that `pandas` has a special-purpose function to parse SQL queries. We can pass in a SQL query as a string to return a `pandas` DataFrame. To achieve the same result as we did using cell magic above, we can do the following.

**1. Connect to a database**

In [ ]:
import sqlalchemy 
import pandas as pd

engine = sqlalchemy.create_engine("sqlite:///data/basic_examples.db")
connection = engine.connect()

**2. Run a simple SQL query**

In [ ]:
query = """
SELECT * 
FROM Dragon;
"""

pd.read_sql(query, engine)

## Tables and Schema

A **database** contains a collection of SQL **tables**. Let's connect to our "toy" database `basic_examples.db` and explore the tables it stores.

In [ ]:
%%sql
sqlite:///data/basic_examples.db

In [ ]:
%%sql
SELECT * FROM sqlite_master WHERE type='table'

## Basic Queries

Every SQL query *must* contain a `SELECT` and `FROM` clause.

* `SELECT`: specify the column(s) to return in the output
* `FROM`: specify the database table from which to extract data

In [ ]:
%%sql
SELECT * FROM Dragon;

In [ ]:
%%sql
SELECT cute, year FROM Dragon;

**Aliasing** with `AS`

In [ ]:
%%sql
SELECT cute AS cuteness,
       year AS birth
FROM Dragon;

**Uniqueness** with `DISTINCT`

In [ ]:
%%sql
SELECT DISTINCT year
FROM Dragon;

**Filtering** with `WHERE`

In [ ]:
%%sql
SELECT name, year
FROM Dragon
WHERE cute > 0;

In [ ]:
%%sql
SELECT name, cute, year
FROM Dragon
WHERE cute > 0 OR year > 2013;

In [ ]:
%%sql
SELECT name, year
FROM Dragon 
WHERE name IN ("puff", "hiccup");

In [ ]:
%%sql
SELECT name, cute
FROM Dragon
WHERE cute IS NOT NULL;

**Ordering** data using `ORDER BY`

In [ ]:
%%sql
SELECT *
FROM Dragon
ORDER BY cute DESC;

**Restricting** output with `LIMIT` and `OFFSET`

In [ ]:
%%sql
SELECT *
FROM Dragon
LIMIT 2;

In [ ]:
%%sql
SELECT *
FROM Dragon
LIMIT 2
OFFSET 1;

## Grouping Data with `GROUP BY`

In [ ]:
%%sql
SELECT *
FROM Dish;

A small note: the fact that `type` is highlighted in green below is a consequence of Jupyter assuming that we are writing Python code (where `type` is a built-in keyword). `type` does *not* have a special meaning in SQL, so the color below does not indicate any special functionality. When we run the cell, Jupyter realizes it should recognize the code as SQL. 

In [ ]:
%%sql
SELECT type
FROM Dish;

In [ ]:
%%sql
SELECT type
FROM Dish
GROUP BY type;

In [ ]:
%%sql
SELECT type, SUM(cost)
FROM Dish
GROUP BY type;

In [ ]:
%%sql
SELECT type, 
       SUM(cost), 
       MIN(cost),
       MAX(name)
FROM Dish
GROUP BY type;

In [ ]:
%%sql
SELECT year, COUNT(cute)
FROM Dragon
GROUP BY year;

In [ ]:
%%sql
SELECT year, COUNT(*)
FROM Dragon
GROUP BY year;

## Working with the `sql` results as Python variables

By default, executing a query with a magic command produces output but doesn't save it into any Python variable:

In [ ]:
%sql SELECT * FROM Dragon

This can be inconvenient if you later want to do further processing of these data in Python.

### Storing one-line `%sql` queries

For simple one-line queries, you can use IPython's ability to store the result of a magic command like `%sql` as if it were any other Python statement, and save the output to a variable:

In [ ]:
dragon_table = %sql SELECT * FROM Dragon
dragon_table

As noted above, the result of the query is a Python variable of type `ResultSet`, more specifically:

In [ ]:
type(dragon_table)

You need to manually convert it to a Pandas DataFrame if you want to do pandas-things with its content:

In [ ]:
dragon_df = dragon_table.DataFrame()
dragon_df

You can configure `jupysql` to _automatically_ convert all outputs to Pandas DataFrames. This can be handy if you intend all your Python-side work to be done with Pandas, as it saves you from manually having to call `.DataFrame()` first on all outputs. On the other hand, you don't get access to the original SQL `ResultSet` object, which have a number of interesting properties and capabilities. You can learn more about those in the [jupysql documentation](https://jupysql.ploomber.io).

For now, let's turn this on so you can see how this simplified, "pandas all the way" worfklow looks like:

In [ ]:
%config SqlMagic.autopandas = True

In [ ]:
dragon_df = %sql SELECT * FROM Dragon
dragon_df

In [ ]:
type(dragon_df)

### Storing multi-line `%%sql` queries

For a more complex query that won't fit in one line, such as for example:

In [ ]:
%%sql
SELECT year, COUNT(*)
FROM Dragon
GROUP BY year;

You can use the `variable <<` syntax in jupysql to store its output (this will honor your `autopandas` state and store either a `sql.run.ResultState` or a Pandas `DataFrame`):

In [ ]:
%%sql dragon_years <<
SELECT year, COUNT(*)
FROM Dragon
GROUP BY year;

In [ ]:
dragon_years

### Filtering Groups Using `HAVING`

In [ ]:
%%sql
SELECT type, COUNT(*)
FROM Dish
GROUP BY type
HAVING MAX(cost) < 8;

In [ ]:
%%sql
SELECT type, MAX(name)
FROM DishDietary
WHERE notes == 'gf'
GROUP BY type
HAVING MAX(cost) <= 7;

### EDA in SQL

Our typical workflow when working with "big data" is:
* Use SQL to query data from a database
* Use Python (with `pandas`) to analyze this data in detail

We can, however, still perform simple data cleaning and re-structuring using SQL directly. To do so, we'll consider the `Title` table from the IMDB dataset.

In [ ]:
%%sql
sqlite:///data/imdbmini.db

In [ ]:
%%sql
SELECT * FROM sqlite_master WHERE type='table'

In [ ]:
%%sql
SELECT *
FROM Title
ORDER BY RANDOM()
LIMIT 10;

#### Matching Text Using `LIKE`

In [ ]:
%%sql
SELECT titleType, primaryTitle
FROM Title
WHERE primaryTitle LIKE "%Star Wars%"

In [ ]:
%%sql
SELECT titleType, primaryTitle
FROM Title
WHERE primaryTitle LIKE "Harry Potter and the Deathly Hallows: Part _"

#### Converting Data Types Using `CAST`

In [ ]:
%%sql
SELECT primaryTitle, CAST(runtimeMinutes AS INT)
FROM Title
LIMIT 10;

### Applying Conditions With `CASE`

Here, we return a random order so we can see the various movie ages (otherwise, the top few entries happen to all be old movies).

In [ ]:
%%sql
SELECT titleType, startYear,
CASE WHEN startYear < 1950 THEN "old"
     WHEN startYear < 2000 THEN "mid-aged"
     ELSE "new"
     END AS movie_age
FROM Title
ORDER BY RANDOM()
LIMIT 10;

### Joining Tables

We combine data from multiple tables by performing a **join**. We will explore joins using the cats database, which includes two tables: `s` and `t`.

In [ ]:
%%sql
sqlite:///data/basic_examples_v2.db

In [ ]:
%%sql
SELECT * FROM s;

In [ ]:
%%sql
SELECT * FROM t;

#### Inner Join

In [ ]:
%%sql
SELECT s.id, name, breed
FROM s INNER JOIN t ON s.id = t.id;

By default, `JOIN`ing without specifying a join type will default to an inner join.

In [ ]:
%%sql
SELECT s.id, name, breed
FROM s JOIN t ON s.id = t.id;

### Cross Join

In [ ]:
%%sql
SELECT *
FROM s CROSS JOIN t;

Equivalently, a cross join can be performed using the following syntax:

In [ ]:
%%sql
SELECT *
FROM s, t;

Conceptually, an inner join is equivalent to a cross join where irrelevant rows are removed.

In [ ]:
%%sql
SELECT *
FROM s CROSS JOIN t
WHERE s.id = t.id;

#### Left Outer Join

In [ ]:
%%sql
SELECT *
FROM s LEFT JOIN t ON s.id = t.id;

#### Right Outer Join

In [ ]:
%%sql
SELECT *
FROM s RIGHT JOIN t ON s.id = t.id;

#### Full Outer Join

In [ ]:
%%sql
SELECT *
FROM s FULL JOIN t ON s.id = t.id;

#### Aliasing in Joins

Let's return to the IMDB dataset. Now, we'll consider two tables: `Title` and `Rating`.

In [ ]:
%%sql
sqlite:///data/imdbmini.db

When working with tables that have long names, we often create an **alias** using the `AS` keyword (much like we did with columns in the previous lecture). This makes it easier to reference these tables when performing a join.

In [ ]:
%%sql
SELECT primaryTitle, averageRating
FROM Title AS T INNER JOIN Rating AS R
ON T.tconst = R.tconst;

The `AS` keyword is optional – omitting it will still create an alias. We typically include `AS` to make our queries more readable.

In [ ]:
%%sql
SELECT primaryTitle, averageRating
FROM Title T INNER JOIN Rating R
ON T.tconst = R.tconst;

Referencing columns using the full or aliased table name is important to avoid ambiguity. Suppose the tables we are trying to join both include a column with the same name, like the `tconst` columns present in both the `Title` and `Rating` tables of the IMDB database. If we do not specify which table's column we wish to reference, SQL will not be able to process our query.

In the cell below, it is unclear if we are referring to the `tconst` column from the `Title` table or the `tconst` column from the `Rating` table. SQL errors.

In [ ]:
%%sql
SELECT primaryTitle, averageRating
FROM Title AS T INNER JOIN Rating AS R
ON tconst = tconst;

Refer to columns in the format `table_alias.column_name` to avoid ambiguity.

In [ ]:
%%sql
SELECT primaryTitle, averageRating
FROM Title AS T INNER JOIN Rating AS R
ON T.tconst = R.tconst;

## Exercise (Submit for participation)

Convert each Pandas expression to SQL assuming data represents a Pandas DataFrame containing 3 columns: name, rank, and year. Both the rank and year are stored as integers.


In [ ]:
data.1oc[data["rank"] < 10, "name"].value_counts().reset_index() 

#Hint:Remember that value counts returns a sorted output!

In [ ]:
data.merge(data,on= "name").sort_values(by="name")

In [ ]:
data.groupby(["name","year"])\
    .filter(lambda sdf: len(sdf)>5)\
    .groupby(["name","year"])["rank"]\
    .min().reset_index().head(5)